In [2]:
import numpy as np
import scipy
from scipy.special import gamma
from scipy.special import beta
import scipy.integrate as integrate

In [14]:
def genParam1(n, randomstate=0) :
    r = np.random.RandomState(seed=randomstate)
    
    alpha = r.exponential(1,(n,3))
    ksi = np.abs(r.standard_cauchy(2))
    beta = [r.dirichlet(alpha[0])]
    for i in range(1,n) :
      betai = [r.dirichlet(alpha[i])]
      beta = np.concatenate((beta,betai), axis=0)
    
    mu = [[beta[0][0]-beta[0][1], 2*(beta[0][0]+beta[0][1])-1]]
    for i in range(1,n) :
      mui = [[beta[i][0]-beta[i][1], 2*(beta[i][0]+beta[i][1])-1]]
      mu = np.concatenate((mu, mui),axis=0)
    sigma = r.wald(ksi[0], ksi[1],n)
    
    return(alpha, ksi, beta, mu, sigma)

In [15]:
def genParam2(n) :
    r = np.random
    
    alpha = r.exponential(1,(n,3))
    ksi = np.abs(r.standard_cauchy(2))
    beta = [r.dirichlet(alpha[0])]
    for i in range(1,n) :
      betai = [r.dirichlet(alpha[i])]
      beta = np.concatenate((beta,betai), axis=0)
    
    mu = [[beta[0][0]-beta[0][1], 2*(beta[0][0]+beta[0][1])-1]]
    for i in range(1,n) :
      mui = [[beta[i][0]-beta[i][1], 2*(beta[i][0]+beta[i][1])-1]]
      mu = np.concatenate((mu, mui),axis=0)
    sigma = r.wald(ksi[0], ksi[1],n)
    
    return(alpha, ksi, beta, mu, sigma)

In [19]:
def genX(n, T, mu, sigma) :
    A = np.eye(T, T+2) + mu[0][0]*np.eye(T, T+2,1) + mu[0][1]*np.eye(T, T+2,2)
    A = A@A.T
    X = [np.random.multivariate_normal(np.zeros(T), sigma[0]*A)]
    for i in range(1,n) :
      A = np.eye(T, T+2) + mu[i][0]*np.eye(T, T+2,1) + mu[i][1]*np.eye(T, T+2,2)
      A = sigma[i]*A@A.T
      Xi = [np.random.multivariate_normal(np.zeros(T), A)]
      X = np.concatenate((X,Xi), axis=0)
    return(X)

In [103]:
PROBA_MUMHprop1 = 1/2
PROBA_MUMHprop2 = 1/2
PROBA_MUMH1 = 1/2
PROBA_MUMH2 = 1/2

T = 100
X = genX(1,T, genParam1(1, 0)[3], genParam1(1, 0)[4])
epsilon = 1

muMH = [np.random.uniform(-1,1,2)]
sigmaMH = [np.abs(np.random.normal(0,1))]

i = 0

for j in range(1) :
    
    AMH = np.eye(T, T+2) + muMH[i][0]*np.eye(T, T+2,1) + muMH[i][1]*np.eye(T, T+2,2)
    AMH = sigmaMH[i]*(AMH@AMH.T)

    muMHprop = np.random.uniform(-1,1,2)
    sigmaMHprop = np.abs(np.random.normal(sigmaMH[i],1))

    AMHprop = np.eye(T, T+2) + muMHprop[0]*np.eye(T, T+2,1) + muMHprop[1]*np.eye(T, T+2,2)
    AMHprop = sigmaMHprop*(AMHprop@AMHprop.T)

    rnum = scipy.stats.multivariate_normal.pdf(X[0], mean = np.zeros(T), cov=AMHprop)*PROBA_MUMHprop1*PROBA_MUMHprop2
    rnum = rnum*scipy.stats.invgauss.pdf(sigmaMHprop, genParam1(1, 0)[1][0]/genParam1(1, 0)[1][1], 0, genParam1(1, 0)[1][1]) 
    rnum = rnum*scipy.stats.uniform.pdf(muMH[i][0],-1,2)*scipy.stats.uniform.pdf(muMH[i][1],-1,2)
    rnum = rnum*scipy.stats.foldnorm.pdf(sigmaMH[i], 0, sigmaMHprop, 1)
    
    rden = scipy.stats.multivariate_normal.pdf(X, mean=np.zeros(T), cov=AMH)*PROBA_MUMH1*PROBA_MUMH2
    rden = rden*scipy.stats.invgauss.pdf(sigmaMH[i], genParam1(1, 0)[1][0]/genParam1(1, 0)[1][1], 0, genParam1(1, 0)[1][1])    
    rden = rden*scipy.stats.uniform.pdf(muMHprop[0],-1,2)*scipy.stats.uniform.pdf(muMHprop[1],-1,2)
    rden = rnum*scipy.stats.foldnorm.pdf(sigmaMHprop, 0, sigmaMH[i], 1)
    
    if rden == 0 :
        r = 0
    else : 
        r = min(1, rnum/rden)
        
    if np.random.uniform()<= r :
        muMH = np.concatenate((muMH, [muMHprop]), axis=0)
        sigmaMH.append(sigmaMHprop)
    else :
        muMH = np.concatenate((muMH, [muMH[i]]), axis=0)
        sigmaMH.append(sigmaMH[i])

    i+=1


2.1661225642730578e-68
6.219017047768435e-69
1.5547542619421088e-69
0.0 0.0


In [55]:
def probmu2(x, u, v) :
  if (x<-1) | (x>1) :
    return(0)
  else :
    return(2*(((x+1)/2)**(u-1))*((1-(x+1)/2)**(v-1))/beta(u,v))

In [ ]:
def probmu1(x, u, v, z) :
  if (z<0) | (z>1-x) :
    return(0)
  else :
    return( (integrate(lambda x : ))   )

In [ ]:
def f(X,theta) :
  psigma = (np.sqrt(theta[4]/(2*np.pi*theta[10]**3)))*np.exp((-theta[4]*(theta[10]-theta[3])**2)/(2*theta[10]*theta[3]**2))
  pksi1 = 2/(np.pi*(1+theta[3]**2))
  pksi2 = 2/(np.pi*(1+theta[4]**2))
  pmu1 =
  pmu2 = probmu2(theta[9],theta[0]+theta[1], theta[2])
  pbeta = (theta[5]**(theta[0]-1))*(theta[6]**(theta[1]-1))*(theta[7]**(theta[1]-1))*gamma(sum(theta[0:3])/(gamma(theta[0])*gamma(theta[1])*gamma(theta[2]))
  palpha = np.exp(-(sum(theta[:3])))

  return(psigma*pksi1*pksi2*pmu1*pmu2*pbeta*palpha)

In [184]:
alpha, ksi, beta, mu, sigma = genParam2(1)
X = genX(1, 50, mu, sigma)

In [185]:
A = np.eye(50, 50+2) + mu[0][0]*np.eye(50, 50+2,1) + mu[0][1]*np.eye(50, 50+2,2)
A = sigma[0]*(A@A.T)

In [188]:
X = genX(1, 50, mu, sigma)

In [191]:
scipy.stats.multivariate_normal.pdf(X, mean=np.zeros(50), cov=A)

5.478668911983611e-34

In [178]:
def Normpdf(X, mean, cov) :
    y =  np.exp(
            (-1/2)*np.dot((X[0]-mean), np.dot(np.linalg.inv(A),(X[0]-mean)))
                         )
    y = y/np.linalg.det(A)**(1/2)
    y = y/(2*np.pi)**(len(X[0])/2)
    
    return(y)

In [192]:
Normpdf(X, np.zeros(len(X[0])), A)

5.478668911983085e-34

In [286]:
ksi = np.abs(np.random.standard_cauchy(2))
sigma = np.random.wald(ksi[0], ksi[1], 100)


In [294]:
ksiMH = [np.abs(np.random.standard_cauchy(2))]
for i in range(0, 1000) :
    L = 1
    ksiMHprop = [np.random.exponential(scale=ksiMH[i][0]), np.random.exponential(scale=ksiMH[i][1])]
    Lprop = 1
    for j in range(len(sigma)):
        L = L*scipy.stats.invgauss.pdf(sigma[j], mu=ksiMH[i][0]/ksiMH[i][1], loc=0, scale=ksiMH[i][1])
        Lprop = Lprop*scipy.stats.invgauss.pdf(sigma[j], mu=ksiMHprop[0]/ksiMHprop[1], loc=0, scale=ksiMHprop[1])
    prior = scipy.stats.halfcauchy.pdf(ksiMH[i][0])*scipy.stats.halfcauchy.pdf(ksiMH[i][1])
    priorprop = scipy.stats.halfcauchy.pdf(ksiMHprop[0])*scipy.stats.halfcauchy.pdf(ksiMHprop[1])
    kernel = scipy.stats.expon.pdf(ksiMH[i][0], scale=ksiMHprop[0])*scipy.stats.expon.pdf(ksiMH[i][1], scale=ksiMHprop[1])
    kernel = kernel/(scipy.stats.expon.pdf(ksiMHprop[0], scale=ksiMH[i][0])*scipy.stats.expon.pdf(ksiMHprop[1], scale=ksiMH[i][1]))
    if L*prior == 0 :
        ksiMH = np.concatenate((ksiMH,[ksiMH[i]]), axis=0)
    else :
        r = min(1, Lprop*priorprop*kernel/(L*prior))
        if np.random.uniform() <= r:
            ksiMH = np.concatenate((ksiMH,[ksiMHprop]), axis=0)
        else : 
            ksiMH = np.concatenate((ksiMH,[ksiMH[i]]), axis=0)


In [223]:
scipy.stats.invgauss.pdf(sigma[j], mu=ksiMH[i][0]/ksiMH[i][1], loc=0, scale=ksiMH[i][1])

nan

In [295]:
m0 = 0
m1 = 0
for i in range(100, ksiMH.shape[0]):
    m0+= ksiMH[i][0]
    m1+= ksiMH[i][1]
m0 = m0/(ksiMH.shape[0]-100)
m1 = m1/(ksiMH.shape[0]-100)